In [2]:
import re
import datetime 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from os.path import join
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error,mean_absolute_percentage_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

DATA_BASE_PATH = '../Data' # Global base path to read data files: set this to the Data/
                           # folder in the local checkout/google drive instance of your
                           # code

In [3]:
def read_and_format_data(base_path=DATA_BASE_PATH):  
    '''
    Read in time series of question responses from the daily_aggregate.pq and 
    initial_indicator_dataset.csv files 
    '''
    # Read daily aggregate file (predictor library)   
    df_agg = pd.read_parquet(join(DATA_BASE_PATH,'daily_aggregate.pq')) 

    # Read indicator data file (target library) 
    df_ind = pd.read_csv(join(DATA_BASE_PATH,'initial_indicator_dataset.csv')).rename(columns={'Unnamed: 0' : 'Date'}) 

    # Process indicator data  
    df_ind['Date'] = pd.to_datetime(df_ind['Date']) # convert dates to datetimes
    df_ind = df_ind.set_index('Date') # set the date column as the index 

    return df_agg, df_ind 

In [4]:
def agg_question(x, treat_as_multiselect=None, gradient=None, uncertainty=None): 
    '''
    Example aggregator function that just take the mean  
    
    Eventually we will want to consider other aggregators here 
    '''
    df_exp = pd.DataFrame(index=x.index)
    # 1. Treat "multiselect" quesitons as default
    if treat_as_multiselect:
        temp = []
        for qid, aid in treat_as_multiselect.items():
            pattern = r'^(' + '|'.join('Q' + str(qid) + 'A' + re.escape(str(id)) for id in aid) + r')'
            data = x.filter(regex=pattern)
            # Check whether this question is in x
            if data.columns.empty:
                continue
            else: 
                temp.append(data)
        df_exp = pd.concat(temp, axis=1)

    if gradient:
        # 2. Aggregate answers by each non-multiselect quesion
        # Select non-multiselect quesion
        for qid, aid in gradient.items():
            data = x.filter(regex='Q' + str(qid))
            
            # Check whether this question is in x
            if data.columns.empty:
                continue
            
            # Drop the answers that are regard as "multi-select"
            if aid:
                lst = ['Q' + str(qid) + 'A' + str(id) for id in aid]
                data = data.drop(columns=lst)

            # Calculate weighted average value of each question, by assigning value 1, 2, 3, ... to each answer respectively
            length = len(data.columns) + 1
            order = np.array(list(range(1, length)))
            data = data @ order
        
            # Standardize
            data /= length
            
            df_exp['Q' + str(qid) + 'Non_Multi'] = data


    if uncertainty:
        # 3. Take the mean of the uncertainty answers
        pattern = r'^(' + '|'.join('Q' + str(qid) + 'A' + re.escape(str(aid)) for qid, aid in uncertainty.items()) + r')'
        data = x.filter(regex=pattern)
        # Check whether this question is in x
        if not data.columns.empty:
            df_exp['uncertainty'] = data.mean(axis=1)


    ## Optional
    # 4. Aggregate expectation value of each question by subcategory
    

    # return np.mean(df_exp,axis=0)
    return df_exp.dropna(axis=1, how='all')
    

In [15]:
def agg_subcategory(subcat, df, pca=False, correlation=False):
    df_sub_agg = pd.DataFrame(index = df.index)

    # Loop through each subcategory
    for qid, aid in subcat.items():
        # Filter out the data under some specific subcategory
        pattern = r'^(' + '|'.join('Q' + re.escape(str(id)) for id in aid) + r')'
        data = df.filter(regex = pattern)

        if correlation:
            print(qid)
            display(data.corr())

        if pca:
            # PCA model require non-NaN data
            if data.isna().any().any():
                raise ValueError('There is NaN in the dataframe!')
            
            # Build the PCA model
            if len(data.columns) > 1:
                pca_model = PCA()
                pca_model.fit(data)
                transformed_data = pca_model.transform(data)
                # choose the component with the largest variance as the aggregation outcome of the subcategory
                df_sub_agg[qid] = pd.DataFrame(transformed_data, index=df.index).iloc[:,0]

            else:
                df_sub_agg[qid] = data
        else:
            # Take the mean as the aggregation outcome of the subcategory
            df_sub_agg[qid] = data.mean(axis=1)
            
    return df_sub_agg

In [6]:
#### File I/0 and initial formatting  
df_agg, df_ind = read_and_format_data() 

In [7]:
treat_as_multiselect = {96: [1109, 1110, 1111], 316: [1820, 1821, 1822, 1823], 597: [1808, 1809, 1810, 1811, 1812, 1813], 664: [1827, 1829, 1830, 1831, 1832, 1833]}
gradient = {97: [], 98: [], 101: [], 103: [], 104: [], 105: [], 106: [], 107: [], 108: [], 109: [], 110: [], 153: [], 154: [], 155: [], 156: [], 158: [], 159: [], 170: [], 173: [], 182: [], 183: [], 316: [1820, 1821, 1822, 1823, 1824], 335: [], 337: [], 338: [], 339: [], 340: [], 341: [], 342: [], 583: [], 584: [], 585: [], 586: [1090], 587: [1093], 595: [], 663: [], 664: [1827, 1829, 1830, 1831, 1832, 1833, 1834], 2400: [], 2401: [], 2402: [], 2403: [], 2406: [], 2428: [], 2429: [3370], 2464: [3671], 2465: [3677], 2466: [3683, 3684], 2467: [3690], 2468: [3695], 2474: [3742]}
uncertainty = {585:1087, 586:1090, 587:1093, 2429:3370, 2464:3671, 2465:3677, 2466:3683, 2467:3690, 2468:3695, 2474:3742}

In [8]:
df_meta = pd.read_parquet('meta.pq')
subcat = dict(df_meta.groupby('subcategory')['questionid'].unique())
subcat

{'Economic Expectations': array([2464, 2465, 2466, 2467, 2468]),
 'Economic Trends': array([338, 339, 340, 341, 342, 583, 584, 585, 586, 587]),
 'Employment': array([ 316,  664, 2406, 2407]),
 'Employment Effects': array([597]),
 'General Shopping Trends': array([182, 183]),
 'Grocery': array([595]),
 'Home Improvement': array([100, 663]),
 'Home Ownership': array([ 96, 101]),
 'Home Value Prediction': array([97, 98]),
 'Inflation': array([2426, 2427, 2428, 2429, 2474]),
 'Online Retail': array([170, 173]),
 'Personal Finances': array([ 335,  337, 2400, 2401, 2402, 2403]),
 'Physical Retail': array([153, 154, 155, 156, 158, 159, 184]),
 'Spending Expectations': array([103, 104, 105, 106, 107, 108, 109, 110])}

In [9]:
df_myagg = agg_question(df_agg, treat_as_multiselect, gradient, uncertainty)
df_myagg.columns

Index(['Q96A1109', 'Q96A1110', 'Q96A1111', 'Q597A1808', 'Q597A1809',
       'Q597A1810', 'Q597A1811', 'Q597A1812', 'Q597A1813', 'Q664A1827',
       'Q664A1829', 'Q664A1830', 'Q664A1831', 'Q664A1832', 'Q664A1833',
       'Q97Non_Multi', 'Q98Non_Multi', 'Q101Non_Multi', 'Q103Non_Multi',
       'Q104Non_Multi', 'Q105Non_Multi', 'Q106Non_Multi', 'Q107Non_Multi',
       'Q108Non_Multi', 'Q109Non_Multi', 'Q110Non_Multi', 'Q153Non_Multi',
       'Q154Non_Multi', 'Q155Non_Multi', 'Q156Non_Multi', 'Q158Non_Multi',
       'Q159Non_Multi', 'Q170Non_Multi', 'Q173Non_Multi', 'Q182Non_Multi',
       'Q183Non_Multi', 'Q335Non_Multi', 'Q337Non_Multi', 'Q338Non_Multi',
       'Q339Non_Multi', 'Q340Non_Multi', 'Q341Non_Multi', 'Q342Non_Multi',
       'Q583Non_Multi', 'Q584Non_Multi', 'Q585Non_Multi', 'Q586Non_Multi',
       'Q587Non_Multi', 'Q595Non_Multi', 'Q663Non_Multi', 'Q664Non_Multi',
       'Q2400Non_Multi', 'Q2401Non_Multi', 'Q2402Non_Multi', 'Q2403Non_Multi',
       'Q2406Non_Multi', 'Q2428Non_

In [10]:
df_myagg

,Q96A1109,Q96A1110,Q96A1111,Q597A1808,Q597A1809,Q597A1810,Q597A1811,Q597A1812,Q597A1813,Q664A1827,...,Q2406Non_Multi,Q2428Non_Multi,Q2429Non_Multi,Q2464Non_Multi,Q2465Non_Multi,Q2466Non_Multi,Q2467Non_Multi,Q2468Non_Multi,Q2474Non_Multi,uncertainty
2019-07-23,0.602623,0.310957,0.086420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-24,0.549287,0.324822,0.125891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-08-19,0.544061,0.298851,0.157088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-08-23,0.619893,0.271758,0.108348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-08-24,0.612205,0.277559,0.110236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,0.596439,0.293769,0.109792,NaN,NaN,NaN,NaN,NaN,NaN,0.010537,...,0.397519,0.319820,0.593994,0.402903,0.304304,0.264765,0.328328,0.321321,0.649249,0.216216
2023-10-05,0.598540,0.299270,0.102190,NaN,NaN,NaN,NaN,NaN,NaN,0.003487,...,0.387216,0.329778,0.597333,0.382222,0.297778,0.251111,0.353333,0.333333,0.660267,0.234133
2023-10-06,0.591029,0.327177,0.081794,NaN,NaN,NaN,NaN,NaN,NaN,0.003660,...,0.399107,0.335335,0.575375,0.417417,0.316316,0.224224,0.329329,0.332733,0.636637,0.216817
2023-10-07,0.590078,0.315927,0.093995,NaN,NaN,NaN,NaN,NaN,NaN,0.003367,...,0.388519,0.333731,0.564200,0.385839,0.281623,0.241448,0.339698,0.337470,0.628162,0.232697


In [16]:
agg_subcategory(subcat, df_myagg, correlation=True)

Economic Expectations


,Q2464Non_Multi,Q2465Non_Multi,Q2466Non_Multi,Q2467Non_Multi,Q2468Non_Multi
Q2464Non_Multi,1.000000,0.778613,0.661176,0.401585,0.264817
Q2465Non_Multi,0.778613,1.000000,0.622007,0.488102,0.269181
Q2466Non_Multi,0.661176,0.622007,1.000000,0.236239,0.083378
Q2467Non_Multi,0.401585,0.488102,0.236239,1.000000,0.829235
Q2468Non_Multi,0.264817,0.269181,0.083378,0.829235,1.000000


Economic Trends


,Q338Non_Multi,Q339Non_Multi,Q340Non_Multi,Q341Non_Multi,Q342Non_Multi,Q583Non_Multi,Q584Non_Multi,Q585Non_Multi,Q586Non_Multi,Q587Non_Multi
Q338Non_Multi,1.000000,0.319843,-0.065658,-0.144567,0.578521,0.417062,0.308547,0.263295,0.719182,0.591643
Q339Non_Multi,0.319843,1.000000,-0.642365,-0.546172,0.299939,-0.309931,-0.455362,-0.315055,0.107252,-0.241539
Q340Non_Multi,-0.065658,-0.642365,1.000000,0.905338,-0.189226,0.684375,0.589158,0.674662,0.239250,0.513861
Q341Non_Multi,-0.144567,-0.546172,0.905338,1.000000,-0.198400,0.631406,0.507021,0.616440,0.174442,0.430868
Q342Non_Multi,0.578521,0.299939,-0.189226,-0.198400,1.000000,0.106229,0.414617,-0.014627,0.394745,0.155551
Q583Non_Multi,0.417062,-0.309931,0.684375,0.631406,0.106229,1.000000,0.692317,0.626267,0.466097,0.723942
Q584Non_Multi,0.308547,-0.455362,0.589158,0.507021,0.414617,0.692317,1.000000,0.469873,0.402309,0.540221
Q585Non_Multi,0.263295,-0.315055,0.674662,0.616440,-0.014627,0.626267,0.469873,1.000000,0.217381,0.436282
Q586Non_Multi,0.719182,0.107252,0.239250,0.174442,0.394745,0.466097,0.402309,0.217381,1.000000,0.676213
Q587Non_Multi,0.591643,-0.241539,0.513861,0.430868,0.155551,0.723942,0.540221,0.436282,0.676213,1.000000


Employment


...

Physical Retail


,Q153Non_Multi,Q154Non_Multi,Q155Non_Multi,Q156Non_Multi,Q158Non_Multi,Q159Non_Multi
Q153Non_Multi,1.000000,0.640555,0.505930,0.464322,0.617794,0.539971
Q154Non_Multi,0.640555,1.000000,0.632428,0.593182,0.585564,0.617558
Q155Non_Multi,0.505930,0.632428,1.000000,0.550777,0.484575,0.551757
Q156Non_Multi,0.464322,0.593182,0.550777,1.000000,0.430782,0.472368
Q158Non_Multi,0.617794,0.585564,0.484575,0.430782,1.000000,0.708089
Q159Non_Multi,0.539971,0.617558,0.551757,0.472368,0.708089,1.000000


Spending Expectations


,Q103Non_Multi,Q104Non_Multi,Q105Non_Multi,Q106Non_Multi,Q107Non_Multi,Q108Non_Multi,Q109Non_Multi,Q110Non_Multi
Q103Non_Multi,1.000000,0.720160,0.838485,0.717767,0.222051,0.297545,-0.533145,0.330005
Q104Non_Multi,0.720160,1.000000,0.793613,0.572806,0.473078,0.542407,-0.328249,0.693198
Q105Non_Multi,0.838485,0.793613,1.000000,0.656086,0.477330,0.551865,-0.399510,0.485338
Q106Non_Multi,0.717767,0.572806,0.656086,1.000000,0.360441,0.344752,-0.303100,0.307818
Q107Non_Multi,0.222051,0.473078,0.477330,0.360441,1.000000,0.752833,0.045594,0.521382
Q108Non_Multi,0.297545,0.542407,0.551865,0.344752,0.752833,1.000000,-0.028161,0.599138
Q109Non_Multi,-0.533145,-0.328249,-0.399510,-0.303100,0.045594,-0.028161,1.000000,-0.020203
Q110Non_Multi,0.330005,0.693198,0.485338,0.307818,0.521382,0.599138,-0.020203,1.000000


,Economic Expectations,Economic Trends,Employment,Employment Effects,General Shopping Trends,Grocery,Home Improvement,Home Ownership,Home Value Prediction,Inflation,Online Retail,Personal Finances,Physical Retail,Spending Expectations
2019-07-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0.318494,NaN,NaN,0.396675,NaN,0.510366
2019-07-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0.330067,NaN,NaN,0.389193,NaN,0.511758
2019-08-19,NaN,0.486617,NaN,NaN,NaN,NaN,NaN,0.333333,0.353873,NaN,NaN,0.371355,NaN,0.520833
2019-08-23,NaN,0.491521,NaN,NaN,NaN,NaN,NaN,0.333333,0.348343,NaN,0.581409,0.395131,0.580923,0.511993
2019-08-24,NaN,0.478416,NaN,NaN,NaN,NaN,NaN,0.333333,0.362179,NaN,0.589489,0.400183,0.584717,0.504095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,0.324324,0.494094,0.124637,NaN,0.370329,0.392697,0.371875,0.425373,0.348881,0.521021,0.461524,0.438151,0.432527,0.512612
2023-10-05,0.323556,0.490200,0.122781,NaN,0.368388,0.393926,0.340608,0.425051,0.353150,0.529126,0.491290,0.442062,0.457994,0.505052
2023-10-06,0.324004,0.496371,0.126112,NaN,0.413615,0.412877,0.340116,0.428013,0.356027,0.515782,0.605215,0.440918,0.583444,0.533627
2023-10-07,0.317216,0.477068,0.120587,NaN,0.395885,0.399858,0.346467,0.427268,0.358407,0.508698,0.585582,0.439827,0.585004,0.535748


In [11]:
agg_subcategory(subcat, df_myagg)

,Economic Expectations,Economic Trends,Employment,Employment Effects,General Shopping Trends,Grocery,Home Improvement,Home Ownership,Home Value Prediction,Inflation,Online Retail,Personal Finances,Physical Retail,Spending Expectations
2019-07-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0.318494,NaN,NaN,0.396675,NaN,0.510366
2019-07-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333333,0.330067,NaN,NaN,0.389193,NaN,0.511758
2019-08-19,NaN,0.486617,NaN,NaN,NaN,NaN,NaN,0.333333,0.353873,NaN,NaN,0.371355,NaN,0.520833
2019-08-23,NaN,0.491521,NaN,NaN,NaN,NaN,NaN,0.333333,0.348343,NaN,0.581409,0.395131,0.580923,0.511993
2019-08-24,NaN,0.478416,NaN,NaN,NaN,NaN,NaN,0.333333,0.362179,NaN,0.589489,0.400183,0.584717,0.504095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,0.324324,0.494094,0.124637,NaN,0.370329,0.392697,0.371875,0.425373,0.348881,0.521021,0.461524,0.438151,0.432527,0.512612
2023-10-05,0.323556,0.490200,0.122781,NaN,0.368388,0.393926,0.340608,0.425051,0.353150,0.529126,0.491290,0.442062,0.457994,0.505052
2023-10-06,0.324004,0.496371,0.126112,NaN,0.413615,0.412877,0.340116,0.428013,0.356027,0.515782,0.605215,0.440918,0.583444,0.533627
2023-10-07,0.317216,0.477068,0.120587,NaN,0.395885,0.399858,0.346467,0.427268,0.358407,0.508698,0.585582,0.439827,0.585004,0.535748


In [12]:
agg_subcategory(subcat, df_myagg.dropna(),pca=True)

,Economic Expectations,Economic Trends,Employment,Employment Effects,General Shopping Trends,Grocery,Home Improvement,Home Ownership,Home Value Prediction,Inflation,Online Retail,Personal Finances,Physical Retail,Spending Expectations
2022-07-29,0.186674,-0.021975,0.081553,0.032935,0.020698,0.417814,0.350962,0.067299,-0.006634,0.083425,0.007496,0.005790,-0.027436,0.006739
2022-07-30,-0.005634,-0.038940,0.068328,0.044520,0.007027,0.414934,0.347299,0.069191,-0.014309,0.002038,0.026897,0.034279,-0.015390,0.015400
2022-07-31,0.004466,-0.035203,0.086577,0.029327,-0.004039,0.409727,0.361328,0.062894,0.006006,-0.000930,-0.000755,0.027973,-0.000218,0.012061
2022-08-01,0.007339,-0.053456,0.023917,-0.030793,0.002037,0.411871,0.362868,0.023185,-0.010628,-0.004228,0.016739,0.013318,0.004517,0.019947
2022-08-02,-0.001281,-0.039344,0.009941,-0.063203,-0.008433,0.418726,0.370614,0.052858,-0.020670,0.031071,0.029341,0.017254,0.022501,0.000139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,-0.025166,-0.008786,0.048050,0.026532,0.026826,0.402015,0.342153,0.019376,0.019359,0.067922,-0.024714,0.030850,-0.015819,0.014535
2023-09-01,-0.026919,0.024224,0.079822,0.063786,0.030037,0.411542,0.351307,-0.059095,0.019756,0.015332,0.015512,0.017269,0.057188,0.019361
2023-09-02,-0.044653,0.028858,0.076209,0.080679,0.003309,0.416710,0.364000,0.016545,0.039403,0.037854,0.029792,0.049328,0.022980,0.001960
2023-09-03,-0.030662,-0.016560,0.041352,0.162846,0.016299,0.395727,0.351601,-0.031308,0.035114,0.030781,0.056725,-0.004596,0.041615,0.020427


In [13]:
agg_subcategory(subcat, df_myagg, pca=True)

ValueError: There is NaN in the dataframe!